In [123]:
#! /usr/bin/env python

import random
import numpy as np
import matplotlib.pyplot as plt
import time

from PIL import Image
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import interact
import ipywidgets as widgets



In [124]:
# define phi as f is at time t = 0 - in particular, phi defines border conditions
def phi(x, y, L):
    if x == 0 or x == L:
        return 1
    return 0
vphi = np.vectorize(phi)

In [125]:

def t_iterations(x, y, L, t, alpha):
    """Simulate the random walk during n iterations.
    If the cemetery is reached, add 0 to the list of values taken.
    Else return the values taken by the walk until n.
    """
    cemeteryReached = False        
    valuesArray = np.array(t*[0])
    for n in range(t): 
        if cemeteryReached: # impossible to leave the cemetery: x and y won't change
            valuesArray[n] = 0
            continue
        if x == 0 or x == L or y == 0 or y == L:
            valuesArray[n] = phi(x, y, L)  # we're on the border: x and y won't change
            continue
        # we are neither on the border nor in the cemetery
        rand = random.random()
        q = (1-alpha)/4
        if 0 <= rand < q:
            x -= 1 # go left
            valuesArray[n] = phi(x, y, L)
        elif q <= rand < 2*q:
            x += 1 # go right
            valuesArray[n] = phi(x, y, L)
        elif 2*q <= rand < 3*q:
            y -= 1 # go down
            valuesArray[n] = phi(x, y, L)
        elif 3*q <= rand < 4*q:
            y += 1 # go up
            valuesArray[n] = phi(x, y, L)
        else: # go to cemetery with probability alpha
            cemeteryReached = True
            valuesArray[n] = 0
    return valuesArray



In [126]:

def monte_carlo(x, y, L, K, t, alpha):    
    """Simulate K times the evolution up to time t starting from point
    (x, y), and return the average result."""
    temporary_sum = np.array(t*[0])
    for _ in range(K):
        temporary_sum += t_iterations(x, y, L, t, alpha)
    return  temporary_sum/K


In [127]:

def approximate_solution(L, K, t, alpha):
    """Approximate solution at time t."""
    averages = np.zeros((t, L, L))
    for x in range(L):
        for y in range(L):
            # The square associated with (x, y) is the one whose bottom left
            # corner is at (x, y)
            averages[:][x][y] = monte_carlo(x, y, L, K, t, alpha) # list of values until 
    return averages



In [128]:
def display(L, K, t, alpha):
    arrayImages = approximate_solution(L, K, t, alpha)
    #print(arrayImages[:][:][1])
    def _show(frame=widgets.IntSlider(min=0,max=np.shape(arrayImages)[0] - 1,step=1,value=0)):
        fig, ax = plt.subplots(figsize=(7,7))
        ax.imshow(arrayImages[frame][:][:].T, origin="lower",
                   extent=[0, 1, 0, 1], cmap="jet")
    interact(_show)

In [129]:
def test():
    """Main function."""
    L = 40  # discretize with squares of length 1/L
    K = 50  # number of simulations per point to compute the average result
    t = 20
    gamma = 0
    alpha = gamma/L**2
    display(L, K, t, alpha)
    
if __name__ == "__main__":
    test()

ValueError: could not broadcast input array from shape (20) into shape (40)